In [1]:
import base64
import IPython
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['Pendulum-v0']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator

In [2]:
py_env = suite_gym.load('Pendulum-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()


TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3,), dtype=tf.float32, name='observation', minimum=array([-1., -1., -8.], dtype=float32), maximum=array([1., 1., 8.], dtype=float32)))

In [3]:
def display_safe_labeling(trajectory):
    label = labeling_functions['Pendulum-v0'](trajectory.observation)
    if tf.reduce_any(label):
        print(label)

In [4]:
replay_buffer_capacity = 1280
# specs
action_spec = tf_env.action_spec()
policy_step_spec = policy_step.PolicyStep(
    action=action_spec,
    state=(),
    info=())
trajectory_spec = trajectory.from_transition(tf_env.time_step_spec(),
                                             policy_step_spec,
                                             tf_env.time_step_spec())

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=trajectory_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)

dataset_generator = lambda: replay_buffer.as_dataset(
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    num_steps=2
).map(
    map_func=lambda trajectory, _: map_rl_trajectory_to_vae_input(trajectory, labeling_function),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    #  deterministic=False  # TF version >= 2.2.0
)

In [5]:
tf_env = tf_py_environment.TFPyEnvironment(py_env)

sac_policy_dir = '../saves/Pendulum-v0/policy/permissive_variance_policy-multiplier=15.0'
policy = tf.compat.v2.saved_model.load(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=5,
                                            observers=[
                                                # display_safe_labeling,
                                                lambda _: py_env.render(mode='human'),
                                                replay_buffer.add_batch
                                            ]).run()

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.95974445,  0.2808747 , -0.7560645 ]], dtype=float32)>),
 ())

In [6]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    num_steps=2
)
iterator = iter(dataset)
trajectory, _ = next(iterator)

state = trajectory.observation[0, ...]
labels = tf.cast(labeling_function(trajectory.observation), tf.float32)
if tf.rank(labels) == 1:
    labels = tf.expand_dims(labels, axis=-1)
label = labels[0, ...]
action = trajectory.action[0, ...]
reward = trajectory.reward[0, ...]
if tf.rank(reward) == 1:
    reward = tf.expand_dims(reward, axis=-1)
next_state = trajectory.observation[1, ...]
next_label = labels[1, ...]

print("\nstate", state)
print('\nlabels', labels)
print('\nlabel', label)
print('\naction', action)
print('\nreward', reward)
print('\nnext_state', next_state)
print('\nnext_label', next_label)


state tf.Tensor([ 0.9263725  -0.3766085  -0.22603792], shape=(3,), dtype=float32)

labels tf.Tensor(
[[1. 1. 0.]
 [1. 1. 0.]], shape=(2, 3), dtype=float32)

label tf.Tensor([1. 1. 0.], shape=(3,), dtype=float32)

action tf.Tensor([0.06637995], shape=(1,), dtype=float32)

reward tf.Tensor(-0.15421203, shape=(), dtype=float32)

next_state tf.Tensor([ 0.91669804 -0.3995807  -0.4985373 ], shape=(3,), dtype=float32)

next_label tf.Tensor([1. 1. 0.], shape=(3,), dtype=float32)


In [7]:
generator = ErgodicMDPTransitionGenerator(labeling_function, replay_buffer)
dataset = replay_buffer.as_dataset(
    num_parallel_calls=4,
    num_steps=2
).map(
    map_func=generator,
    num_parallel_calls=4,
    #  deterministic=False  # TF version >= 2.2.0
).batch(batch_size=8, drop_remainder=True)
iterator = iter(dataset)
next(iterator)

(<tf.Tensor: shape=(8, 3), dtype=float32, numpy=
 array([[ 6.4294201e-01, -7.6591486e-01, -1.5775427e+00],
        [ 9.6671438e-01, -2.5585803e-01,  8.1465445e-02],
        [-7.7596229e-01, -6.3077927e-01,  6.2149205e+00],
        [ 9.4504619e-01,  3.2693693e-01,  9.9686503e-02],
        [ 9.7825205e-01,  2.0741959e-01, -3.3456117e-03],
        [ 9.1297662e-01,  4.0801185e-01,  2.7977255e-01],
        [ 9.5500523e-01,  2.9658905e-01, -1.6795512e-01],
        [ 9.6319401e-01,  2.6880711e-01,  2.4182208e-01]], dtype=float32)>,
 <tf.Tensor: shape=(8, 4), dtype=float32, numpy=
 array([[1., 0., 0., 0.],
        [1., 1., 1., 0.],
        [0., 0., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(8, 1), dtype=float32, numpy=
 array([[ 1.955008 ],
        [ 1.5483292],
        [ 0.9434242],
        [-1.9972148],
        [-1.9793999],
        [-0.299812 ],
        [-1.9

In [5]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    # "../../saves/Pendulum-v0/models/vae_LS12_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA3_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step140000/eval_elbo-3.784"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA5_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step270000/eval_elbo-1.704"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/permissive_variance_policy-multiplier=10.0/action_discretizer/LA5_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step70000/eval_elbo-7.265"
    # "../../saves/Pendulum-v0/models/vae_LS12_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.95-0.90_1e-06-2e-06/policy/action_discretizer/LA3_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step1010000/eval_elbo11.174"
    # '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.95-0.90_1e-06-2e-06/policy/action_discretizer/LA3_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step1010000/eval_elbo-2.888'
    # '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.70-0.57_2e-06-1e-06_seed=290321/policy/action_discretizer/LA3_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.50-0.33_2e-06-1e-06_params=full_vae_optimization-relaxed_state_encoding/step360000/eval_elbo4.516'
    '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e-06-2e-06_seed=20421/policy/action_discretizer/LA3_MC1_ER10.0-decay=7.5e-05-min=0_KLA0.0-growth=5e-05_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization-relaxed_state_encoding/base',
    step=1830000
)
print("VAE MDP loaded")

discrete_tf_env = vae_mdp.wrap_tf_environment(
    tf_env=tf_env,
    labeling_function=labeling_function,
    deterministic_embedding_functions=True
)
discrete_tf_env.reset()
print(discrete_tf_env.time_step_spec(), discrete_tf_env.action_spec())

reward_metric = tf_metrics.AverageReturnMetric()

policy = vae_mdp.get_latent_policy()

dynamic_episode_driver.DynamicEpisodeDriver(
    discrete_tf_env,
    policy,
    num_episodes=30,
    observers=[
        lambda _: py_env.render(mode='human'),
        reward_metric
    ]
).run()

print("avg rewards", reward_metric.result())

VAE MDP loaded
TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(12,), dtype=tf.int32, name='observation', minimum=array(0, dtype=int32), maximum=array(1, dtype=int32))) BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32))
avg rewards tf.Tensor(-673.27014, shape=(), dtype=float32)


In [5]:
name = '../../saves/Pendulum-v0/models/vae_LS12_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.70-0.57_2e-06-1e-06_seed=290321/policy/action_discretizer/LA3_MC1_ER10.0-decay=0.0001-min=0_KLA0.0-growth=5e-06_TD0.50-0.33_2e-06-1e-06_params=full_vae_optimization-relaxed_state_encoding/step350000/eval_elbo4.750'
model = tf.saved_model.load(name)

In [7]:
model

In [ ]:
import imageio
import IPython

def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

def create_policy_eval_video(policy, py_env, tf_env, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = tf_env.reset()
      print(py_env.render(mode='rgb_array'))
      video.append_data(py_env.render(mode='rgb_array'))
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render(mode='rgb_array'))
  return embed_mp4(filename)

create_policy_eval_video(policy, py_env, tf_env, 'pendulum-latent-policy')